In [343]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
	using Flux: @functor
	using ..Transformers: Abstract3DTensor, Container, epsilon, batchedmul, batched_triu!
	using Einsum
end
	

In [344]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end

Vocabulary{String}(13, unk=0)

In [345]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [346]:
#nichts ändern

sample_data()

(["2", "5", "8", "8", "10", "4", "4", "2", "9", "1"], ["2", "5", "8", "8", "10", "4", "4", "2", "9", "1"])

In [347]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [348]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data
    end

sample_ex = preprocess.(sample_data()) = (["11", "7", "4", "2", "10", "7", "8", "4", "9", "7", "9", "12"], ["11", "7", "4", "2", "10", "7", "8", "4", "9", "7", "9", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 10, 7, 5, 13, 10, 11, 7, 12, 10, 12, 3]


12-element Vector{Int64}:
  2
 10
  7
  5
 13
 10
 11
  7
 12
 10
 12
  3

In [349]:
#nichts ändern
sample = preprocess.(sample_data())

(["11", "5", "10", "2", "2", "6", "9", "2", "3", "7", "9", "12"], ["11", "5", "10", "2", "2", "6", "9", "2", "3", "7", "9", "12"])

In [350]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
  8
 13
  5
  5
  9
 12
  5
  6
 10
 12
  3

In [351]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

Embed(512)

In [352]:
#define a position embedding layer metioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu

PositionEmbedding(512)

In [353]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [354]:
abstract type AbstractAttention end

In [355]:
struct MultiheadAttention_1{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end

In [356]:
Flux.functor(la::LinearAttention) = (la.iqproj, la.ikproj, la.ivproj, la.oproj), m -> LinearAttention(la.head, la.future, m..., la.drop)


In [357]:
LinearAttention(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = LinearAttention(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, la::LinearAttention)
    hs = div(size(la.iqproj.weight)[1], la.head)
    is = size(la.iqproj.weight)[end]
    os = size(la.oproj.weight)[1]

    print(io, "LinearAttention(")
    print(io, "head=$(la.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(la.drop.p))")
    else
        print(io, ")")
    end
end

In [358]:
# of type float (to allow for integer inputs)
oftf(x, y) = oftype(float(x), y)
nelu(x, α=1) = ifelse(x ≥ 0, float(x)+1, @fastmath oftf(x, α) * (exp(x) - 1)+1)

nelu (generic function with 2 methods)

In [359]:
function splitHeads(x, batch_size, head, depth)
    x = reshape(x, (batch_size, :, head, depth))
end

splitHeads (generic function with 2 methods)

In [360]:
function (la::LinearAttention)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}

#size(query) == (dims, {q,k}_seq_len, batch) == size(key) == size(value)
#size(ipq) == (hs, q_seq_len, head * batch)
#size(score) == (k_seq_len, q_seq_len, batch)
#=
dk = size(key, 1)
score = batchedmul(key, query; transA = true) 
score = score ./ convert(T, sqrt(dk))

score = apply_mask1(score, mask, future)
score = softmax(score; dims=1)
dropout !== nothing && (score = dropout(score))
batchedmul(value, score) #size(return) == (dims, q_seq_len, batch)
=#
#size(query) == (batch, {q,k}_seq_len, dims) == size(key) == size(value)
query = permutedims(query,(3,2,1))
key = permutedims(key,(3,2,1))
value = permutedims(value,(3,2,1))

qs = size(query)
ks = size(key)
vs = size(value)

#depth = div(la.size, la.head)
depth = div(512, 8)   

#query = nelu(splitHeads(query, qs[1], la.head, depth))
#key = nelu(splitHeads(key, qs[1], la.head, depth))
#value = splitHeads(value, qs[1], la.head, depth)

query = nelu(splitHeads(query, qs[1], 8, depth))
key = nelu(splitHeads(key, ks[1], 8, depth))
value = splitHeads(value, vs[1], 8, depth)
# size(k_v) == (batch_size, depth_k, depth_v, seq_len_v)
@einsum k_v[m,d,e,h] := key[m,j,h,d]*value[m,j,h,e]

k_reduced = sum(key, dims=2) .+ 1e-8

@einsum z_1[m,l,h] := query[m,l,h,d]*k_reduced[m,h,d]
# size(z) == (batch_size, num_heads, seq_len_q)
z = 1/z_1 # ...
# size(output) == (batch_size,len_q, heads, depth_v)
@einsum output[m,l,h,e] := query[m,l,h,d]*k_v[m,d,e,h]*z[m,l,h]

#output = reshape(output, (qs[1], qs[2], la.head*depth))
# size(output) == (batch_size,len_q, d_model)
output = reshape(output, (qs[1], :, 8*depth))

output = permutedims(output,(3,2,1))

out = @toNd la.oproj(output)
#(dims, q_seq_len, batch)
end

function (la::LinearAttention)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: AbstractMatrix{T},
                         A2 <: AbstractMatrix{T},
                         A3 <: AbstractMatrix{T}}

#size(query) == (dims, {q,k}_seq_len, batch) == size(key) == size(value)
#size(ipq) == (hs, q_seq_len, head * batch)
#size(score) == (k_seq_len, q_seq_len, batch)
#=
dk = size(key, 1)
score = batchedmul(key, query; transA = true) 
score = score ./ convert(T, sqrt(dk))

score = apply_mask1(score, mask, future)
score = softmax(score; dims=1)
dropout !== nothing && (score = dropout(score))
batchedmul(value, score) #size(return) == (dims, q_seq_len, batch)
=#
#size(query) == (batch, {q,k}_seq_len, dims) == size(key) == size(value)
query = permutedims(query,(3,2,1))
key = permutedims(key,(3,2,1))
value = permutedims(value,(3,2,1))

qs = size(query)
ks = size(key)
vs = size(value)

#depth = div(la.size, la.head)
depth = div(512, 8)   

#query = nelu(splitHeads(query, qs[1], la.head, depth))
#key = nelu(splitHeads(key, qs[1], la.head, depth))
#value = splitHeads(value, qs[1], la.head, depth)

query = nelu(splitHeads(query, qs[1], 8, depth))
key = nelu(splitHeads(key, ks[1], 8, depth))
value = splitHeads(value, vs[1], 8, depth)
# size(k_v) == (batch_size, depth_k, depth_v, seq_len_v)
@einsum k_v[m,d,e,h] := key[m,j,h,d]*value[m,j,h,e]

k_reduced = sum(key, dims=2) .+ 1e-8

@einsum z_1[m,l,h] := query[m,l,h,d]*k_reduced[m,h,d]
# size(z) == (batch_size, num_heads, seq_len_q)
z = 1/z_1 # ...
# size(output) == (batch_size,len_q, heads, depth_v)
@einsum output[m,l,h,e] := query[m,l,h,d]*k_v[m,d,e,h]*z[m,l,h]

#output = reshape(output, (qs[1], qs[2], la.head*depth))
# size(output) == (batch_size,len_q, d_model)
output = reshape(output, (qs[1], :, 8*depth))

output = permutedims(output,(3,2,1))
#(dims, q_seq_len, batch)

la.oproj(output)
end

In [361]:
abstract type AbstractTransformer end

In [362]:
struct PwFFN{Di<:Dense, Do<:Dense}
    din::Di
    dout::Do
end

@functor PwFFN


"just a wrapper for two dense layer."
PwFFN(size::Int, h::Int, act = nelu) = PwFFN(  #relu vielleicht auf elu + 1 oder nur elu ändern
    Dense(size, h, act),
    Dense(h, size)
)

function (pw::PwFFN)(x::AbstractMatrix)
  # size(x) == (dims, seq_len)
  pw.dout(pw.din(x))
end

function (pw::PwFFN)(x::A) where {T, N, A<:AbstractArray{T, N}}
  new_x = reshape(x, size(x, 1), :)
  y = pw(new_x)
  return reshape(y, Base.setindex(size(x), size(y, 1), 1))
end

In [363]:
struct Transformer1{MA<:LinearAttention, LA<:LayerNorm, P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
    la::MA
    mhn::LA
    pw::P
    pwn::LP
    drop::DP
end

@functor Transformer1


"""
    Transformer(size::Int, head::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)
    Transformer(size::Int, head::Int, hs::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)  

Transformer layer.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
When `future` is `false`, the k-th token can't see the j-th tokens where j > k. `pdrop` is the dropout rate.
"""

########
#ganze Funktion von Transformer ändern
########
function Transformer1(size::Int, head::Int, ps::Int; future::Bool = true, act = nelu, pdrop = 0.1)  #relu vielleicht wieder auf elu ändern
    rem(size, head) != 0 && error("size not divisible by head")
    Transformer1(size, head, div(size, head), ps;future=future, act=act, pdrop=pdrop)
end

Transformer1(size::Int, head::Int, hs::Int, ps::Int; future::Bool = true, act = nelu, pdrop = 0.1) = Transformer1(
    LinearAttention(head, size, hs, size; future=future, pdrop=pdrop),
    LayerNorm(size),
    PwFFN(size, ps, act),
    LayerNorm(size),
    Dropout(pdrop),     #braucht man das? vielleicht nicht
)

function (t::Transformer1)(x::A, mask=nothing) where {T, N, A<:AbstractArray{T, N}}
    dropout = t.drop
    a = t.la(x, x, x; mask=mask)
    a = dropout(a)
    res_a = x + a
    res_a = t.mhn(res_a)
    pwffn = t.pw(res_a)
    pwffn = dropout(pwffn)
    res_pwffn = res_a + pwffn
    res_pwffn = t.pwn(res_pwffn)
    res_pwffn
end

function Base.show(io::IO, t::Transformer1) #zum visualisieren --> nice to have
    hs = div(size(t.la.iqproj.weight)[1], t.la.head)
    h, ps = size(t.pw.dout.weight)

    print(io, "Transformer(")
    print(io, "head=$(t.la.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "pwffn_size=$(ps), ")
    print(io, "size=$(h)")
    if Flux.istraining()
        print(io, ", dropout=$(t.drop.p))")
    else
        print(io, ")")
    end
end


ErrorException: invalid redefinition of constant Transformer1

In [364]:
###########
    #auch diese Funktion ändern
    ###############
    struct TransformerDecoder1{MA<:LinearAttention, LA<:LayerNorm,
        IMA<:LinearAttention, ILA<:LayerNorm,
        P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
la::MA
mhn::LA
ila::IMA
imhn::ILA
pw::P
pwn::LP
drop::DP
end

@functor TransformerDecoder1

"""
TransformerDecoder(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
TransformerDecoder(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1)

TransformerDecoder layer. Decode the value from a Encoder.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
`pdrop` is the dropout rate.
"""
function TransformerDecoder1(size::Int, head::Int, ps::Int; act = nelu, pdrop = 0.1)
rem(size, head) != 0 && error("size not divisible by head")
TransformerDecoder1(size, head, div(size, head), ps; act=act, pdrop=pdrop)
end

TransformerDecoder1(size::Int, head::Int, hs::Int, ps::Int; act = nelu, pdrop = 0.1) = TransformerDecoder1(
LinearAttention(head, size, hs, size; future=false, pdrop=pdrop),
LayerNorm(size),
LinearAttention(head, size, hs, size; future=true, pdrop=pdrop), #future = true --> Unterschied zu oben??
LayerNorm(size),
PwFFN(size, ps, act),
LayerNorm(size),
Dropout(pdrop),
)

function (td::TransformerDecoder1)(x::AbstractArray{T,N}, m, mask=nothing) where {T,N}
dropout = td.drop
a = td.la(x,x,x)
a = dropout(a)
res_a = x + a
res_a = td.mhn(res_a)

ia = td.ila(res_a, m, m, mask=mask)
ia = dropout(ia)
res_ia = res_a + ia
res_ia = td.imhn(res_ia)

pwffn = td.pw(res_ia)
pwffn = dropout(pwffn)
res_pwffn = res_ia + pwffn
res_pwffn = td.pwn(res_pwffn)
res_pwffn
end

function Base.show(io::IO, td::TransformerDecoder1)
hs = div(size(td.ila.iqproj.weight)[1], td.ila.head)
h, ps = size(td.pw.dout.weight)

print(io, "TransformerDecoder(")
print(io, "head=$(td.la.head), ")
print(io, "head_size=$(hs), ")
print(io, "pwffn_size=$(ps), ")
print(io, "size=$(h)")
if Flux.istraining()
print(io, ", dropout=$(td.drop.p))")
else
print(io, ")")
end
end

ErrorException: invalid redefinition of constant TransformerDecoder1

In [365]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [366]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [367]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [368]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    return t1
  end

encoder_forward (generic function with 1 method)

In [369]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [370]:
#bleibt so
enc = encoder_forward(encoded_sample)

ArgumentError: ArgumentError: no valid permutation of dimensions

In [371]:
#bleibt so
probs = decoder_forward(encoded_sample, enc)

UndefVarError: UndefVarError: enc not defined

In [372]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [373]:
Flux.@nograd smooth

In [374]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])  #erstmal so lassen
    return l
  end

loss (generic function with 1 method)

In [375]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, decode_t1, linear)

Params([Float32[-0.51304907 -0.6683715 … -0.80315036 0.5276025; -0.39588508 -0.55599225 … 0.14842303 -1.3252602; … ; -0.5312266 0.25819048 … 0.32759124 1.7346628; 0.7313382 -2.0601692 … -0.47881743 -1.3610063], Float32[0.5403023 -0.41614684 … 0.4000682 0.9873536; 0.8218562 0.9364147 … 0.37902638 0.97646356; … ; 1.0 1.0 … 0.9943822 0.99437124; 0.0001 0.0002 … 0.10212166 0.10222114], Float32[-0.046199363 0.045024894 … -0.063788846 -0.042573355; 0.056413334 0.04221744 … -0.011564555 0.06920331; … ; -0.066903815 -0.05477294 … 0.05219281 0.06633255; -0.02792442 0.05107262 … 0.06759633 0.0073322235], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.05767657 -0.027813094 … -0.05032909 0.06377225; -0.0108496975 0.064189985 … -0.0142692225 0.03808293; … ; -0.016464567 0.024370208 … 0.015878372 -0.0021333296; 0.06784048 0.024988431 … 0.028356856 -0.07620384], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0,

In [376]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [377]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:100
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [378]:
train!()

MethodError: MethodError: no method matching isless(::Int64, ::Array{Float32, 4})
Closest candidates are:
  isless(::Integer, !Matched::ForwardDiff.Dual{Ty, V, N} where {V, N}) where Ty at /Users/johannes/.julia/packages/ForwardDiff/PBzup/src/dual.jl:145
  isless(::Real, !Matched::ColorTypes.AbstractGray{T} where T) at /Users/johannes/.julia/packages/ColorTypes/6m8P7/src/operations.jl:32
  isless(::Real, !Matched::ForwardDiff.Dual{Ty, V, N} where {V, N}) where Ty at /Users/johannes/.julia/packages/ForwardDiff/PBzup/src/dual.jl:145
  ...

In [379]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

┌ Info: start training
└ @ Main /Users/johannes/Documents/GitHub/DM2022-LinearTransformers/test/LinearAttentionTransformer.ipynb:5


translate (generic function with 1 method)

In [380]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

ArgumentError: ArgumentError: no valid permutation of dimensions